<a href="https://colab.research.google.com/github/AhmedSaleh171/Thesis/blob/main/Uberset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_csv('https://raw.githubusercontent.com/AhmedSaleh171/Thesis/refs/heads/main/Uber_dynamic_pricing.csv')
df.head()





,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422


#Encoding

In [56]:
df['Vehicle_Type'] = df['Vehicle_Type'].apply(lambda x: 1 if x == 'Premium' else 0).astype(int)

one_hot_columns = ['Location_Category', 'Customer_Loyalty_Status', 'Time_of_Booking']
one_hot_encoded = pd.get_dummies(df[one_hot_columns], prefix=one_hot_columns)
one_hot_encoded = one_hot_encoded.astype(int)
df= pd.concat([df,one_hot_encoded], axis=1)
df= df.drop(columns=one_hot_columns)
df.head()

,Number_of_Riders,Number_of_Drivers,Number_of_Past_Rides,Average_Ratings,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Location_Category_Rural,Location_Category_Suburban,Location_Category_Urban,Customer_Loyalty_Status_Gold,Customer_Loyalty_Status_Regular,Customer_Loyalty_Status_Silver,Time_of_Booking_Afternoon,Time_of_Booking_Evening,Time_of_Booking_Morning,Time_of_Booking_Night
0,90,45,13,4.47,1,90,284.257273,0,0,1,0,0,1,0,0,0,1
1,58,39,72,4.06,0,43,173.874753,0,1,0,0,0,1,0,1,0,0
2,42,31,0,3.99,1,76,329.795469,1,0,0,0,0,1,1,0,0,0
3,89,28,67,4.31,1,134,470.201232,1,0,0,0,1,0,1,0,0,0
4,78,22,74,3.77,0,149,579.681422,1,0,0,0,1,0,1,0,0,0


#Model

In [57]:
X = df.drop(columns=['Historical_Cost_of_Ride'])
y = df['Historical_Cost_of_Ride']

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)



#Evaluation

In [58]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmse_perc = (rmse / np.mean(y_test)) * 100
mae_perc = (mae / np.mean(y_test)) * 100

print(f"Root Mean Squared Error (RMSE): {rmse_perc:.2f}%")
print(f"Mean Absolute Error (MAE): {mae_perc:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Root Mean Squared Error (RMSE): 18.86%
Mean Absolute Error (MAE): 14.19%
Mean Absolute Percentage Error (MAPE): 15.03%
